---
---
## Only for testing:

In [1]:
%load_ext autoreload
%autoreload 2
%pylab inline
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from harps import salinity_harps as sh

Populating the interactive namespace from numpy and matplotlib


In [13]:
data = sh.read_harp_file('salinity_harps_test_data.dat')
# data = data.sel(module=0)
resistance = data['r2']

read salinity harp now... done


In [17]:
# smoothen the signal
resistance_smooth = sh.median(resistance, time=100)
data['r16_smooth'] = resistance_smooth
# compute gradient of that smoothened signal
resistance_grad = sh.median(sh.grad(resistance_smooth, 20))
data['r16_grad'] = resistance_grad


# set all values below a certain threshold to nan and remove them
condition = resistance_grad > 1e-4
grad_tol = resistance_grad.where(~condition, 9999)

# alternative:
# condition = resistance_grad > 1e-4
# grad_tol = resistance_grad.where(condition, drop=True)

# then find the index of the first occurrence
freezing_starts = grad_tol.argmax('time')
# alternative:
# freezing_starts = grad_tol.argmin('time')
data['freezing_start'] = data.time[freezing_starts].where(freezing_starts > 0)


In [ ]:
grad_tol.max()

In [18]:
grad_tol.sel(module=0).to_pandas().plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:xlabel='time'>

In [19]:
%matplotlib widget

d = data.sel(module=0)
p = d.r16_smooth.to_pandas().plot(logy=True, figsize=(15,10))
# d.r16.to_pandas().plot(logy=True, ax=plt.gca())
plt.vlines(d.freezing_start.values, d.r16_smooth.min(), d.r16_smooth.max(), colors=plt.rcParams['axes.prop_cycle'].by_key()['color'])


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
resistance_grad.sel(module=0).to_pandas().plot(logy=False)
resistance_grad.sel(module=0).to_pandas().plot(logy=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:xlabel='time'>

In [7]:
resistance_grad.roll(time=0).sel(module=0).to_pandas().plot()

/home/mpim/m300660/miniconda3/envs/co2/lib/python3.6/site-packages/xarray/core/dataarray.py:2979: FutureWarning: roll_coords will be set to False in the future. Explicitly set roll_coords to silence warning.
  shifts=shifts, roll_coords=roll_coords, **shifts_kwargs


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:xlabel='time'>

In [9]:
data.temperature.sel(module=0).to_pandas().plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:xlabel='time'>

In [57]:
data

<xarray.Dataset>
Dimensions:         (module: 2, time: 4999, wire_pair: 8)
Coordinates:
  * time            (time) datetime64[ns] 2018-12-23T11:41:11 ... 2019-01-02T...
  * module          (module) int64 0 1
  * wire_pair       (wire_pair) int64 0 1 2 3 4 5 6 7
Data variables:
    d16             (time, module, wire_pair) float64 2.0 2.0 2.0 ... nan nan
    d2              (time, module, wire_pair) float64 2.0 2.0 2.0 ... nan nan
    logger_temp     (time, module, wire_pair) float64 -16.33 -16.33 ... nan nan
    r16             (time, module, wire_pair) float64 4.741 4.783 ... nan nan
    r2              (time, module, wire_pair) float64 7.886 7.885 ... nan nan
    temperature     (time, module, wire_pair) float64 0.5 1.375 ... nan nan
    r16_smooth      (time, module, wire_pair) float64 nan nan nan ... nan nan
    r16_grad        (time, module, wire_pair) float64 nan nan nan ... nan nan
    freezing_start  (module, wire_pair) datetime64[ns] 2018-12-24T05:03:06 .....
Attributes:
    Temperature:  °C
    Resistance:   Ohm